In [1]:
import pandas as pd
import os

In [2]:
# Ottiene la directory di lavoro corrente
script_dir = os.getcwd()

# Costruisce il percorso relativo al file CSV
file_path = os.path.join(script_dir, "main_data", "studios.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "clean_data")

# Crea la cartella clean_data se non esiste
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Controlla che il file esista prima di caricarlo
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Il file non esiste: {file_path}")

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)
df.head()

,id,studio
0,1000001,LuckyChap Entertainment
1,1000001,Heyday Films
2,1000001,NB/GG Pictures
3,1000001,Mattel
4,1000001,Warner Bros. Pictures


In [3]:
df = df[df["studio"] != "."]

In [4]:
df = df.apply(lambda col: col.str.lstrip('/') if col.dtypes == "object" else col)

In [5]:
from unidecode import unidecode

df = df.apply(lambda col: col.map(lambda x: unidecode(str(x))) if col.dtypes == "object" else col)

In [6]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "studios_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

# Collegamento al db

In [7]:
import psycopg2
from sqlalchemy import create_engine, text

In [8]:
#  Parametri di connessione PostgreSQL
DB_NAME = "ium_database"  
DB_USER = "postgres"       
DB_PASSWORD = "password"   
DB_HOST = "localhost"      
DB_PORT = "5432"    

conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) # Crea una connessione al database usando psycopg2.connect
conn.autocommit = True # Conferma automaticamente ogni operazione evitando la conferma manuale dopo ogni query
cur = conn.cursor() # Crea un oggetto cur che permette di eseguire query SQL sul database
cur.execute(f"CREATE DATABASE {DB_NAME};") # Esegue il comando per creare il database con il nome specificato

DuplicateDatabase: ERRORE:  il database "ium_database" esiste già


In [9]:
# Crea un oggetto di connessione a un database PostgreSQL utilizzando SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [10]:
# Query SQL per creare la tabella
create_table_query = text("""
    CREATE TABLE IF NOT EXISTS actors_data (
        id INT,
        name TEXT,
        role TEXT
    );
""")

# Eseguire la query
with engine.connect() as connection: # Apre una connessione al database usando SQLAlchemy
    connection.execute(create_table_query) # Esegue il comando per la creazione della 
    connection.commit() # Salva le modifiche

In [11]:
# Legge il CSV pulito
df = pd.read_csv(cleaned_csv_path)

# Carica i dati nella tabella PostgreSQL
df.to_sql("studios_data", engine, if_exists="replace", index=False) # Esporta il dataframe nel database, sovrascrive e aggiunge i nuovi dati alla tabella esistente

278